In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

2021-12-20 10:06:34.435430: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-20 10:06:34.435938: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv("train.csv")

In [3]:
df = df.drop(['qid'],axis=1)
df.head(10)

,question_text,target
0,What is the role of Lua in Civ4?,0
1,What are important chapters in Kannada for 10 ...,0
2,Do musicians get royalties from YouTube?,0
3,What is the difference between Scaling Social ...,0
4,Why do elevators go super slow right before th...,0
5,Could the Jewish mafia control certain scienti...,0
6,What are the disadvantages of making lasagna w...,0
7,All charges were withdrawn by the state siting...,0
8,Why was supercharger in Mad Max fake?,0
9,Is phone hacking common in Finland?,0


In [4]:
df["target"].value_counts()

0    612656
1     40405
Name: target, dtype: int64

In [5]:
sentiment_label = df.target.factorize()
sentiment_label

(array([0, 0, 0, ..., 0, 0, 0]), Int64Index([0, 1], dtype='int64'))

In [6]:
text = df.question_text.values
print(len(text))

653061


In [7]:
tokenizer = Tokenizer(num_words=3000)
tokenizer.fit_on_texts(text)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(text)
padded_sequence = pad_sequences(encoded_docs, maxlen=200)

In [8]:
print(text[0])
print(encoded_docs[0])
print(padded_sequence[0])

What is the role of Lua in Civ4?
[2, 3, 1, 518, 7, 6]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   2   3   1 518
   7   6]


In [15]:
embedding_vector_length = 32
model = Sequential() 
model.add(Embedding(vocab_size, embedding_vector_length, input_length=200) )
model.add(SpatialDropout1D(0.25))
model.add(LSTM(50, dropout=0.5, recurrent_dropout=0.5))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])  
print(model.summary()) 

2021-12-20 09:54:07.820574: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-20 09:54:07.821716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-20 09:54:07.821793: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-20 09:54:07.821855: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-20 09:54:07.821916: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 32)           4850976   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                16600     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 4,867,627
Trainable params: 4,867,627
Non-trainable params: 0
______________________________________________

In [16]:
history = model.fit(padded_sequence,sentiment_label[0],validation_split=0.2, epochs=5, batch_size=32)

2021-12-20 09:55:25.248933: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 417958400 exceeds 10% of free system memory.


Epoch 1/5
  518/16327 [..............................] - ETA: 43:32 - loss: 0.2365 - accuracy: 0.9379

KeyboardInterrupt: 